In [ ]:
%%capture
!pip install transformers
!pip install evaluate
!pip install datasets
!pip install accelerate -U

In [ ]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, AutoConfig, TrainingArguments, DataCollatorWithPadding
import numpy as np
import evaluate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = "/content/drive/MyDrive/Federated_learning/Train_split/"
validation_path = "/content/drive/MyDrive/Federated_learning/Validation_split/"
model_save_path = "/content/drive/MyDrive/Federated_learning/Federated_models/estimate_trainer/"

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer
import torch

In [ ]:
num_models = 8

models = []
for model_number in range(num_models):
  models.append(AutoModelForSequenceClassification.from_pretrained(model_save_path+ str(model_number)))

In [ ]:
federated_model = AutoModelForSequenceClassification.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
print(federated_model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12,

In [ ]:
#Init
federated_model = AutoModelForSequenceClassification.from_pretrained("microsoft/MiniLM-L12-H384-uncased")

federated_model.classifier.bias.data = torch.zeros(federated_model.classifier.bias.data.size())
federated_model.classifier.weight.data = torch.zeros(federated_model.classifier.weight.data.size())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
federated_model.classifier.weight.data

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [ ]:
#adding every model weight to the fed model
for model in models:
  federated_model.classifier.bias.data = torch.add(federated_model.classifier.bias.data, model.classifier.bias.data)
  federated_model.classifier.weight.data = torch.add(federated_model.classifier.weight.data, model.classifier.weight.data)

In [ ]:
federated_model.classifier.weight.data

tensor([[-2.4082e-01,  8.4334e-02, -1.2741e-01, -7.6641e-03, -6.1510e-02,
          2.5640e-02, -8.2543e-02,  1.3340e-01,  4.0631e-01, -1.5986e-01,
         -1.0595e-01,  1.9310e-01,  1.5693e-01, -2.3348e-01,  1.8939e-01,
         -9.6069e-02,  2.8349e-03,  2.6836e-01,  3.2205e-01, -2.2138e-01,
          2.7267e-02,  8.9435e-02, -3.8074e-02, -1.2473e-01, -1.3992e-02,
         -8.2221e-02,  7.0425e-02, -1.6648e-01,  1.4813e-01, -3.8020e-01,
          1.4638e-01,  6.4010e-02,  2.1239e-01, -4.2385e-02,  1.9647e-01,
         -1.7192e-01,  2.1495e-01,  2.1325e-01, -3.4959e-02,  2.6457e-01,
         -1.8841e-01,  1.3305e-01, -1.0036e-01,  1.5597e-01,  2.4612e-01,
         -2.2425e-01, -2.3749e-01, -1.1093e-01,  2.0706e-01, -7.9309e-02,
          9.2009e-03, -1.4531e-01,  7.1304e-02, -1.9345e-01, -1.2727e-01,
          9.9770e-02,  2.7818e-01, -2.7149e-02, -2.9892e-01, -7.0380e-02,
         -9.1110e-03,  1.5348e-02, -5.1334e-01,  3.7738e-02, -1.5767e-01,
          1.2119e-01,  9.5184e-02, -1.

In [ ]:
federated_model.classifier.bias.data = torch.div(federated_model.classifier.bias.data, num_models)
federated_model.classifier.weight.data = torch.div(federated_model.classifier.weight.data, num_models)

In [ ]:
federated_model.classifier.weight.data

tensor([[-3.0102e-02,  1.0542e-02, -1.5926e-02, -9.5801e-04, -7.6887e-03,
          3.2050e-03, -1.0318e-02,  1.6675e-02,  5.0789e-02, -1.9982e-02,
         -1.3244e-02,  2.4138e-02,  1.9616e-02, -2.9185e-02,  2.3674e-02,
         -1.2009e-02,  3.5437e-04,  3.3545e-02,  4.0257e-02, -2.7672e-02,
          3.4084e-03,  1.1179e-02, -4.7593e-03, -1.5591e-02, -1.7490e-03,
         -1.0278e-02,  8.8031e-03, -2.0810e-02,  1.8516e-02, -4.7525e-02,
          1.8298e-02,  8.0013e-03,  2.6549e-02, -5.2981e-03,  2.4558e-02,
         -2.1490e-02,  2.6869e-02,  2.6656e-02, -4.3699e-03,  3.3071e-02,
         -2.3551e-02,  1.6631e-02, -1.2545e-02,  1.9497e-02,  3.0765e-02,
         -2.8031e-02, -2.9687e-02, -1.3867e-02,  2.5882e-02, -9.9137e-03,
          1.1501e-03, -1.8164e-02,  8.9130e-03, -2.4181e-02, -1.5908e-02,
          1.2471e-02,  3.4773e-02, -3.3936e-03, -3.7366e-02, -8.7975e-03,
         -1.1389e-03,  1.9185e-03, -6.4168e-02,  4.7173e-03, -1.9709e-02,
          1.5148e-02,  1.1898e-02, -2.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")

raw_datasets = load_dataset("glue", "mrpc")


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"])


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



training_args = TrainingArguments(model_save_path+"Federated_trainer/", num_train_epochs=70, learning_rate=1e-5,
                                lr_scheduler_type="linear", per_device_train_batch_size=16, seed=42,
                                per_device_eval_batch_size=16)

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    federated_model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [2]:
predictions = trainer.predict(tokenized_datasets["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.6589427368257387, 'f1': 8078254825485992}
